In [5]:
import requests
import base64
import json

# Your OpenRouter API key (replace with yours)
API_KEY = "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"  # Get this from openrouter.ai

# Free vision model (supports images and is free within daily limits)
MODEL = "meta-llama/llama-3.2-11b-vision-instruct"

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Main function to get description
def get_image_description(image_path):
    # Encode the image
    base64_image = encode_image(image_path)

    # API endpoint
    url = "https://openrouter.ai/api/v1/chat/completions"

    # Headers
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Payload (like OpenAI's chat API)
    payload = {
        "model": MODEL,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Describe this apartment space in detail, focusing on features like rooms, swimming pools, balcony, attached bathrooms, and other amenities visible in the image."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"  # Use jpeg or change to png if needed
                        }
                    }
                ]
            }
        ]
    }

    # Send request
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    # Parse and return description
    if response.status_code == 200:
        result = response.json()
        description = result['choices'][0]['message']['content']
        return description
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage: Run this in VSCode terminal with `python image_describer.py`
if __name__ == "__main__":
    # image_path = input("Enter the path to your image (e.g., /path/to/your/image.jpg): ")
    image_path = r"C:\Users\MSI\Downloads\webscrap\1.jpg"
    description = get_image_description(image_path)
    print("\nGenerated Description:\n")
    print(description)


Generated Description:

The image depicts a living room with a gray couch, two framed pictures on the wall, and a coffee table. The room is decorated with plants and has a large window with gray curtains.

* **Gray Couch:**
	+ Location: Against the back wall of the room
	+ Color: Light gray
	+ Size: L-shaped, with one section facing the window and the other facing the coffee table
	+ Decorations: Two throw pillows and a throw blanket
* **Two Framed Pictures:**
	+ Location: Above the couch, one on either side of the window
	+ Size: Large, rectangular frames
	+ Content: One picture appears to be a close-up of a fish's tail, while the other is a floating staircase
	+ Style: Modern, minimalist
* **Coffee Table:**
	+ Location: In front of the couch
	+ Shape: Round, with a flat top and a curved base
	+ Color: Light brown wood
	+ Decorations: A small potted plant on top
* **Plants:**
	+ Location: On both sides of the couch and around the room
	+ Type: Various types of green plants in white p

In [23]:
# Improved version with better error handling and fallback options
import requests
import base64
import json
import os
import time

# Your OpenRouter API key
API_KEY = "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"

# Models to try
VISION_MODEL = "meta-llama/llama-3.2-11b-vision-instruct"
TEXT_MODEL = "meta-llama/llama-3.1-8b-instruct:free"

def encode_image(image_path):
    """Encode image to base64"""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding {image_path}: {e}")
        return None

def get_single_image_description(image_path, max_retries=3):
    """Get description for a single image with retry logic"""
    for attempt in range(max_retries):
        try:
            # print(f"  Attempt {attempt + 1} for {os.path.basename(image_path)}")
            
            # Encode the image
            base64_image = encode_image(image_path)
            if not base64_image:
                return f"Failed to encode image: {os.path.basename(image_path)}"
            
            # Determine mime type
            filename = os.path.basename(image_path).lower()
            mime_type = "image/png" if filename.endswith(".png") else "image/jpeg"

            # API endpoint
            url = "https://openrouter.ai/api/v1/chat/completions"

            # Headers
            headers = {
                "Authorization": f"Bearer {API_KEY}",
                "Content-Type": "application/json"
            }

            # Payload for single image
            payload = {
                "model": VISION_MODEL,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": "Describe this apartment/room image in detail. Focus on: rooms, furniture, layout, style, colors, amenities, and any notable features. Be specific and descriptive."
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:{mime_type};base64,{base64_image}"
                                }
                            }
                        ]
                    }
                ],
                "max_tokens": 500,
                "temperature": 0.7
            }

            # Send request
            response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                if 'choices' in result and len(result['choices']) > 0:
                    content = result['choices'][0]['message']['content']
                    # Filter out suspicious responses
                    if len(content) > 20 and not content.startswith("safesafesafe"):
                        return content
                    else:
                        print(f"    Suspicious response, retrying...")
                        continue
                else:
                    print(f"    No choices in response: {result}")
                    if attempt == max_retries - 1:
                        return f"No valid response for {os.path.basename(image_path)}"
            else:
                print(f"    HTTP {response.status_code}: {response.text}")
                if attempt == max_retries - 1:
                    return f"API Error for {os.path.basename(image_path)}: {response.status_code}"
            
            # Wait before retry
            if attempt < max_retries - 1:
                time.sleep(2)
                
        except Exception as e:
            print(f"    Exception: {e}")
            if attempt == max_retries - 1:
                return f"Error processing {os.path.basename(image_path)}: {str(e)}"
            time.sleep(2)
    
    return f"Failed to process {os.path.basename(image_path)} after {max_retries} attempts"

def create_simple_combined_description(descriptions):
    """Create a dynamic, natural description for sublease websites"""
    try:
        # Filter out error messages and suspicious responses
        valid_descriptions = []
        for desc in descriptions:
            if not any(error_phrase in desc.lower() for error_phrase in 
                      ['error', 'failed', 'safesafesafe', 'suspicious response']):
                valid_descriptions.append(desc)
        
        if not valid_descriptions:
            return "No valid descriptions were generated from the images."
        
        # Extract key features and amenities from all descriptions
        key_features = []
        room_types = []
        amenities = []
        colors = []
        styles = []
        
        # Keywords to look for in descriptions
        room_keywords = ['bedroom', 'bathroom', 'kitchen', 'living room', 'balcony', 'dining room', 'office', 'study', 'closet']
        amenity_keywords = ['pool', 'swimming pool', 'gym', 'fitness', 'parking', 'laundry', 'dishwasher', 
                           'microwave', 'refrigerator', 'air conditioning', 'heating', 'fireplace',
                           'hardwood floors', 'tile', 'carpet', 'granite', 'stainless steel', 'walk-in closet']
        feature_keywords = ['spacious', 'modern', 'updated', 'renovated', 'bright', 'natural light',
                           'open concept', 'high ceilings', 'large windows', 'private', 'quiet', 'cozy', 'elegant']
        color_keywords = ['white', 'black', 'brown', 'beige', 'gray', 'grey', 'wooden', 'dark', 'light']
        style_keywords = ['contemporary', 'traditional', 'minimalist', 'rustic', 'industrial', 'classic']
        
        # Analyze all descriptions
        combined_text = ' '.join(valid_descriptions).lower()
        
        # Extract various elements
        for keyword in room_keywords:
            if keyword in combined_text:
                room_types.append(keyword)
        
        for keyword in amenity_keywords:
            if keyword in combined_text:
                amenities.append(keyword)
        
        for keyword in feature_keywords:
            if keyword in combined_text:
                key_features.append(keyword)
                
        for keyword in color_keywords:
            if keyword in combined_text:
                colors.append(keyword)
                
        for keyword in style_keywords:
            if keyword in combined_text:
                styles.append(keyword)
        
        # Dynamic opening variations
        opening_variations = []
        if key_features and len(key_features) > 0:
            opening_variations.append(f"Discover this {key_features[0]} apartment")
            opening_variations.append(f"Step into this {key_features[0]} living space")
        if colors and len(colors) > 0:
            opening_variations.append(f"Experience this {colors[0]} themed residence")
        if styles and len(styles) > 0:
            opening_variations.append(f"Embrace {styles[0]} living in this apartment")
        
        # Default openings if no specific features found
        if not opening_variations:
            opening_variations = [
    "Welcome to this inviting apartment",
    "Experience comfortable living in this unit",
    "Discover your new home in this residence",
    "Step into your dream apartment",
    "Explore modern elegance in this home",
    "Find your perfect retreat in this space",
    "Uncover stylish living in this apartment",
    "Welcome to your urban oasis",
    "Experience the charm of this residence",
    "Discover contemporary comfort in this unit",
    "Make this stunning apartment your home",
    "Enter a world of cozy sophistication",
    "Your ideal living space awaits"
]
        
        # Choose opening based on content hash for consistency
        opening = opening_variations[abs(hash(combined_text)) % len(opening_variations)]
        description = opening
        
        # Add room information dynamically
        if room_types:
            unique_rooms = list(dict.fromkeys(room_types))  # Remove duplicates while preserving order
            if len(unique_rooms) == 1:
                description += f" featuring a thoughtfully designed {unique_rooms[0]}"
            elif len(unique_rooms) == 2:
                description += f" with a {unique_rooms[0]} and {unique_rooms[1]}"
            else:
                description += f" boasting {', '.join(unique_rooms[:-1])}, and a {unique_rooms[-1]}"
        
        # Add features dynamically
        if key_features:
            selected_features = key_features[:2]  # Limit to 2 features
            if len(selected_features) == 1:
                description += f" showcasing {selected_features[0]} design elements"
            else:
                description += f" enhanced by {' and '.join(selected_features)} touches"
        
        description += ". "
        
        # Add amenities dynamically
        if amenities:
            unique_amenities = list(dict.fromkeys(amenities))[:3]  # Remove duplicates, limit to 3
            amenity_intros = ["Equipped with", "Features include", "Highlights include", "Amenities feature"]
            intro = amenity_intros[abs(hash(''.join(unique_amenities))) % len(amenity_intros)]
            
            if len(unique_amenities) == 1:
                description += f"{intro} {unique_amenities[0]}"
            elif len(unique_amenities) == 2:
                description += f"{intro} {unique_amenities[0]} and {unique_amenities[1]}"
            else:
                description += f"{intro} {', '.join(unique_amenities[:-1])}, and {unique_amenities[-1]}"
            description += ". "
        
        # Dynamic closing statements
        target_audiences = []
        if 'study' in room_types or 'office' in room_types:
            target_audiences.append("students and remote workers")
        elif 'modern' in key_features or 'contemporary' in styles:
            target_audiences.append("young professionals")
        elif 'spacious' in key_features or 'large' in combined_text:
            target_audiences.append("those seeking extra space")
        else:
            target_audiences.append("anyone looking for quality living")
        
        closing_variations = [
        f"Ideal for {target_audiences[0] if target_audiences else 'comfortable living'}",
        f"Well-suited for {target_audiences[0] if target_audiences else 'modern lifestyles'}",
        f"Perfect match for {target_audiences[0] if target_audiences else 'discerning renters'}",
        f"Designed for {target_audiences[0] if target_audiences else 'vibrant living'}",
        f"Tailored to {target_audiences[0] if target_audiences else 'dynamic professionals'}",
        f"Perfect for {target_audiences[0] if target_audiences else 'those seeking luxury'}",
        f"Great choice for {target_audiences[0] if target_audiences else 'stylish residents'}",
        f"Crafted for {target_audiences[0] if target_audiences else 'urban dwellers'}",
        f"Suitable for {target_audiences[0] if target_audiences else 'cozy retreats'}",
        f"Best fit for {target_audiences[0] if target_audiences else 'elegant living'}"
    ]
        action_phrases = [
        "Available for immediate sublease",
        "Ready for your next chapter",
        "Your next home awaits",
        "Now available for sublease",
        "Move-in ready today",
        "Waiting for you to call home",
        "Secure your spot now",
        "Available to lease immediately",
        "Your new home is ready",
        "Start your journey here"
    ]
        
        contact_phrases = [
    "Schedule your visit today",
    "Contact us for more details",
    "Reach out to arrange a viewing",
    "Get in touch to secure this opportunity",
    "Book a tour now",
    "Inquire today to learn more",
    "Call us to schedule a visit",
    "Connect with us for details",
    "Arrange your private tour today",
    "Reach out to make it yours"
]
        
        # Select phrases based on content for consistency
        closing = closing_variations[abs(hash(combined_text[:50])) % len(closing_variations)]
        action = action_phrases[abs(hash(combined_text[50:100] if len(combined_text) > 50 else combined_text)) % len(action_phrases)]
        contact = contact_phrases[abs(hash(combined_text[-50:] if len(combined_text) > 50 else combined_text)) % len(contact_phrases)]
        
        description += f"{closing}. {action}. {contact}!"
        
        return description
        
    except Exception as e:
        return f"Error creating combined description: {str(e)}"

def process_apartment_images(image_dir):
    """Main function to process all images in directory"""
    try:
        # Supported image extensions
        valid_extensions = (".jpg", ".jpeg", ".png")
        descriptions = []
        
        # print("Processing images individually...\n")
        
        # Get all image files
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(valid_extensions)]
        
        if not image_files:
            return "No valid images found in the directory."
        
        # print(f"Found {len(image_files)} image(s): {', '.join(image_files)}\n")
        
        # Process each image
        for filename in image_files:
            image_path = os.path.join(image_dir, filename)
            # print(f"Processing: {filename}")
            
            description = get_single_image_description(image_path)
            descriptions.append(description)
            # print(f"✓ Completed: {filename}")
            # print(f"  Preview: {description[:100]}...\n")
        
        # Create combined description
        # print("Creating combined apartment description...")
        combined_description = create_simple_combined_description(descriptions)
        
        return combined_description
        
    except Exception as e:
        return f"Error processing apartment images: {str(e)}"

# Run the improved version
if __name__ == "__main__":
    image_dir = r"C:\Users\MSI\Downloads\webscrap\images"
    
    # print("APARTMENT IMAGE ANALYZER")
    # print("=" * 50)
    # print(f"Processing images from: {image_dir}\n")
    
    final_description = process_apartment_images(image_dir)
    
    print("=" * 60)
    print("FINAL APARTMENT DESCRIPTION:")
    print("=" * 60)
    print(final_description)

FINAL APARTMENT DESCRIPTION:
Experience this white themed residence boasting bedroom, kitchen, and a balcony enhanced by spacious and modern touches. Features include pool, refrigerator, and tile. Tailored to young professionals. Your new home is ready. Call us to schedule a visit!
